# Libraries

In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
%run ../../../OpenAI_API.ipynb


The openai.ChatCompletion.create() function is used to generate a response to a sequence of messages in the context of a conversation. Here are the parameters of the function:

* model: The ID of the GPT model to use for generating the response. This can be a string representing the name of the model, or an instance of the openai.Model class.
* prompt: An optional string containing the initial prompt to start the conversation. This can be used to set the context for the conversation.
* temperature: A float specifying the "creativity" of the generated responses. Higher values result in more diverse and unexpected responses.
* max_tokens: An integer specifying the maximum number of tokens (words and punctuation) that the generated response should contain.
* n: An integer specifying the number of responses to generate. The API will return the top n results.
* stop: An optional string or list of strings specifying the stopping criteria for the generated response. When the generated text c

# Static parameters
These parameters are hand-made and relevant only for the Climatebase.org website. It is assumed that a major redesign labor would be required to adapt the code for another website.

In [3]:
#https://climatebase.org/jobs?l=&q=&p=0&remote=false
domain_name = "https://climatebase.org"
url_path = "/jobs?l=&q=&p=0&remote=false"

# Job types
#https://climatebase.org/jobs?l=&q=&job_types=Full+time+role&p=0&remote=false
d_job_types  = {0:"", 1:"Full+time+role", 2:"Internship"}

# Role type
#https://climatebase.org/jobs?l=&q=&categories=Data+Analyst&p=0&remote=false
d_categories = {0:"", 1:"Data+Analyst", 2:"Data+Scientist", 3:"Research"}

# Remote
#https://climatebase.org/jobs?l=Remote&q=&p=0&remote=true
d_remote = {0:"", 1:"true", 2:"false"}

css_object_class = "list_card"

# User-Defined Functions

In [4]:
def insert_filter(input_text, to_insert):
    """
    This function formats the url structure to make a filtered query.
    """
    # Find the index where "&p=" starts
    index = input_text.find("&p=")

    # Insert the text to the left of "&p="
    new_string = input_text[:index] + to_insert + input_text[index:]
    
    return new_string

In [5]:
def define_remote(input_text):
    """
    This function is similar to insert_filter(), but is specific for the "remote" filtering.
    """
    
    new_string = input_text.replace("?l=", "?l=Remote")
    new_string = input_text.replace("&remote=false", "&remote=true")

    return new_string

In [6]:
def scraping_css_object(url_path, css_object_class):
    """
    Given a CSS object class, this scraper will obtain the relevant information from the website.
    """
    
    url = domain_name + url_path
    #"https://climatebase.org/jobs?l=&q=&categories=Data+Scientist&p=0&remote=true"

    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")

    # Find all elements with class="list_card"
    found_objects = soup.find_all(class_=css_object_class)

    return found_objects

# User-defined parameters

In [7]:
# These parameters are the filtering criteria for the website.

job_types = d_job_types[1]
print("Job type: " + job_types.replace("+", ""))

categories = d_categories[1]
print("Category: " + categories.replace("+", ""))

remote = d_remote[1]
print("Remote: " + remote)


Job type: Fulltimerole
Category: DataAnalyst
Remote: true


# Variables

In [8]:
# Formatting variables for filtering criteria on the website.

if job_types != "":
    url_path = insert_filter(url_path, "&job_types=" + job_types)
    
if categories != "":
    url_path = insert_filter(url_path, "&categories=" + categories)

if remote != "":    
    url_path = define_remote(url_path)

# Mining

## > Mining url's
Mining url's from main site by filtered criteria. 

In [9]:
# After applying the filtering criteria, s
scraped_url_paths = [element['href'] for element in scraping_css_object(url_path, css_object_class)]
#scraped_url_paths

# Visualization of the complete url
#domain_name + scraped_url_paths[0]

## > Mining information
Title and job description are obtained from one url.

In [11]:
current_path = scraped_url_paths[0]

In [12]:
# Mining job title
html_title = scraping_css_object(current_path, "fcPVcr")

soup = BeautifulSoup(str(html_title), 'html.parser')
title = soup.find('h1', {'class': 'PageLayout__Title-sc-1ri9r3s-4 fcPVcr'}).text

print(title)

Mid/Senior Product Designer


In [13]:
# Mining job description
html_bodytext = scraping_css_object(current_path, "EPUZp")

soup = BeautifulSoup(str(html_bodytext), 'html.parser')
bodytext = soup.div.text.strip()

print(bodytext)

Do you want to design part of the clean energy grid of the future?
Granular is a fast-growing climate tech startup developing a platform to help electricity consumers, producers and suppliers move towards 24/7 clean energy. Our SaaS platform gives our clients visibility over how electricity was produced on each hour using hourly energy certificates and allow them to trade clean energy with each other. You can find out more about the 24/7 energy space in this article.
We are active across Europe and the US and have partnered with Europe’s leading power exchange and grid operators, among others. Our seed round was led by some of the world’s top early-stage VCs, and we are currently preparing for our next funding round.
What will I be doing
We are looking for a Senior Product Designer to join our growing tech team. You will actively participate in the product development and delivery.
In a typical day you will

Have a meaningful impact on the decarbonisation of the grid
Work alongside our

In [19]:
text_prompt = """I will prompt you with a job description and I want your help to categorize it, but before I will set some rules. 

1. Please categorize the job description by the following criteria (if the information is available): 
* Job title
* Company mission
* Company values 
* Company products or services
* Job responsibilities
* Desired software skills
* Education
* Required Job Experience
* Equal Employment Opportunity
* Salary
* Benefits
* Location
* Type of employment

Please provide your answers in a JSON object format, where the key name is the same as the categories but with spaces replaced by underscores (if necessary).
If any category has no available information, please include a "null" value for the corresponding key in the JSON object.
4. Make the categorizations as concise as possible, maybe even as keywords. Be as economic as possible.
5. Avoid paragraphs of text or long sentences. 
6. Avoid redundant text.

Those would be the rules. Now I will prompt you with the text for the job description: 
""" + "Job title: " + title + "."+ bodytext

reply = call_openai_api(text_prompt, tokens = 1000)

In [20]:
reply_back = reply


In [21]:
reply = reply.replace("\n", "") 
reply = reply.replace("'", "\"")

In [22]:
dict_obj = json.loads(reply)
dict_obj


{'Job_title': 'Mid/Senior Product Designer',
 'Company_mission': 'Developing a platform to help electricity consumers, producers and suppliers move towards 24/7 clean energy',
 'Company_values': None,
 'Company_products_or_services': 'SaaS platform for trading clean energy',
 'Job_responsibilities': ['Design and refine the UI/UX of the Granular web platform',
  'Contribute to the design system and maintain design consistency across features',
  'Lead user research, testing, and validation of new designs',
  'Contribute to ideation along with the founders and the rest of the technology and product team',
  'Support with occasional brand work'],
 'Desired_software_skills': ['Figma', 'Prototyping'],
 'Education': None,
 'Required_Job_Experience': '4+ years of designing user-centric products – anything that is data-heavy or relating to energy is a big plus',
 'Equal_Employment_Opportunity': 'We are committed to diversity and developing talent. If you do not have all the skills listed above

In [18]:
reply_back

'{\n  "Job_title": "Mid/Senior Product Designer",\n  "Company_mission": "Developing a platform to help electricity consumers, producers and suppliers move towards 24/7 clean energy.",\n  "Company_products_or_services": "SaaS platform that gives clients visibility over how electricity was produced on each hour using hourly energy certificates and allows them to trade clean energy with each other.",\n  "Job_responsibilities": "Design and refine the UI/UX of the Granular web platform, contribute to the design system and maintain design consistency across features, lead user research, testing, and validation of new designs, contribute to ideation along with the founders and the rest of the technology and product team, support with occasional brand work.",\n  "Desired_software_skills": "Figma, its component and prototyping system.",\n  "Education": "null",\n  "Required_job_experience": "4+ years of designing user-centric products – anything that is data-heavy or relating to energy is a big 